### Data preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
trainingData = pd.read_csv('../../data/trainData.csv', header=None)
# AATI = Average Access Time Interval
trainingData.columns = ["timestamp", "sector_id", "# of blocks"]
trainingData.head()

# trainingData.Frequency.hist(bins=15)

,timestamp,sector_id,# of blocks
0,1.652817,7487488,2048
1,1.652824,7489536,2048
2,1.652830,7491584,2048
3,1.652836,7493632,2048
4,1.652842,7495680,2048


In [3]:
testData = pd.read_csv("../../data/testData.csv", header=None)
testData.columns = ["timestamp", "sector_id", "# of blocks"]
testData.head()

print(testData.head())

   timestamp  sector_id  # of blocks
0   0.000000     303567            7
1   0.000000      55590            6
2   0.026214     303574            7
3   0.026214     240840            6
4   0.117964     303581            7


In [4]:
testDataLabel = pd.read_csv("../../data/lableing/testDataLabeled.csv", header=None)
testDataLabel.columns = ["sector_id", "frequency", "AATI", "# of blocks","hot/cold"]
print(testDataLabel.head())

   sector_id  frequency      AATI  # of blocks  hot/cold
0     753921      90736  0.459198       544416         1
1     240840      48735  0.886414       292465         1
2     836706      31787  1.296780       195293         1
3     837306      31704  1.299350       192217         1
4     700132      31288  1.156710       247313         1


In [5]:
trainDataLabel = pd.read_csv("../../data/lableing/trainDataLabeled.csv", header=None)
trainDataLabel.columns = ["sector_id", "frequency", "AATI", "# of blocks","hot/cold"]
print(trainDataLabel.head())

   sector_id  frequency     AATI  # of blocks  hot/cold
0       8488        966  36.0808         7728         1
1     205888        948  36.7295         7584         1
2     206064        948  36.7666         7584         1
3      74328        947  36.7683         7576         1
4      74408        945  36.8834         7560         1


In [6]:
# Constants
trainLabelSize = trainDataLabel["sector_id"].size
print("trainDataLabel size:", trainLabelSize)

trainDataLabel size: 1586700


In [7]:
trainDataLabel.dtypes

sector_id        int64
frequency        int64
AATI           float64
# of blocks      int64
hot/cold         int64
dtype: object

In [8]:
trainingData.dtypes

timestamp      float64
sector_id        int64
# of blocks      int64
dtype: object

In [9]:
trainingYLabelMap = {}

for i in range(trainLabelSize):
    trainingYLabelMap[trainDataLabel["sector_id"][i]] = trainDataLabel["hot/cold"][i]

trainingData["hot/cold"] = [trainingYLabelMap[sectorId] if sectorId in trainingYLabelMap else np.nan for sectorId in trainingData["sector_id"]]


In [10]:
testLabelSize = testDataLabel["sector_id"].size
testYLabelMap = {}

for i in range(testLabelSize):
    testYLabelMap[testDataLabel["sector_id"][i]] = testDataLabel["hot/cold"][i]

testData["hot/cold"] = [testYLabelMap[sectorId] if sectorId in testYLabelMap else np.nan for sectorId in testData["sector_id"]]

In [11]:
trainDataLabel["hot/cold"].value_counts()

0    1354099
1     232601
Name: hot/cold, dtype: int64

In [12]:
trainingData["hot/cold"].value_counts()


1    6057980
0    3628281
Name: hot/cold, dtype: int64

In [13]:
testData["hot/cold"].value_counts()

1    2070048
0    2029306
Name: hot/cold, dtype: int64

In [14]:
# Normalizing number of blocks
trainBlocksMean = trainingData["# of blocks"].mean()
trainBlocksStd = trainingData["# of blocks"].std()

trainingData["# of blocks"] = (trainingData["# of blocks"] - trainBlocksMean) / trainBlocksStd

testBlocksMean = testData["# of blocks"].mean()
testBlocksStd = testData["# of blocks"].std()

testData["# of blocks"] = (testData["# of blocks"] - testBlocksMean) / testBlocksStd

In [15]:
# Try univariate LSTM first
features = trainingData[["sector_id"]].to_numpy().tolist()
target = trainingData[["hot/cold"]].to_numpy().tolist()

In [16]:
features[0:5]


[[7487488], [7489536], [7491584], [7493632], [7495680]]

In [17]:
target[0:11]

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]

### Model Implementation

In [18]:
import tensorflow as tf

2022-07-26 11:28:52.369503: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-26 11:28:53.016510: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-07-26 11:28:54.203531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-07-26 11:28:54.203747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [21]:
# Note: I basically concluded that large sector_ids are not possible to embed(preprocess).
# However, I thought of a way to represent sector_ids in tensors and that is one hot encoding and deep neural networks (aka. Dense, Fully connected)
# It could be wrong and I want more research on 
#   **representing numbers in one-hot encoding and RNN**
#   **Is it possible to feed LSTM large numbers**
# also checkout tf.data.Dataset.grouping_window() and tf.data.Dataset.window() functions 
# i think they can be used to generate windows. gl
ds = tf.data.Dataset.from_tensor_slices((features, target))
ds = ds.window(10, shift=1, drop_remainder=True)

<WindowDataset element_spec=(DatasetSpec(TensorSpec(shape=(1,), dtype=tf.int32, name=None), TensorShape([])), DatasetSpec(TensorSpec(shape=(1,), dtype=tf.int32, name=None), TensorShape([])))>

In [39]:
# Operations on window
count = 0

def to_numpy(ds):
    return list(ds.as_numpy_iterator())

for window in ds:
    if count == 5:
        break
    print(to_numpy(window[0]))
    count += 1

[array([7487488], dtype=int32), array([7489536], dtype=int32), array([7491584], dtype=int32), array([7493632], dtype=int32), array([7495680], dtype=int32), array([7497728], dtype=int32), array([7499776], dtype=int32), array([7501824], dtype=int32), array([7503872], dtype=int32), array([7505920], dtype=int32)]
[array([7489536], dtype=int32), array([7491584], dtype=int32), array([7493632], dtype=int32), array([7495680], dtype=int32), array([7497728], dtype=int32), array([7499776], dtype=int32), array([7501824], dtype=int32), array([7503872], dtype=int32), array([7505920], dtype=int32), array([7507968], dtype=int32)]
[array([7491584], dtype=int32), array([7493632], dtype=int32), array([7495680], dtype=int32), array([7497728], dtype=int32), array([7499776], dtype=int32), array([7501824], dtype=int32), array([7503872], dtype=int32), array([7505920], dtype=int32), array([7507968], dtype=int32), array([7510016], dtype=int32)]
[array([7493632], dtype=int32), array([7495680], dtype=int32), arra

2022-07-26 11:55:02.932733: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled


In [20]:
maxSectorNumber = np.amax(trainingData["sector_id"])
maxBlocks = np.amax(trainingData["# of blocks"])
print("maxSectorNumber:", maxSectorNumber)
print("maxBlocks:", maxBlocks)

maxSectorNumber: 1000213824
maxBlocks: 0.6759364762275851


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [27]:
trainingSectorSize = trainLabelSize
trainLabelSize

1586700

In [23]:
model = tf.keras.Sequential()

# embedding the sector number from 0-1
model.add(tf.keras.layers.Embedding(input_dim=maxSectorNumber+1,output_dim=64))

model.add(tf.keras.layers.LSTM(64,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

2022-07-26 11:39:35.713593: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 238.47GiB (rounded to 256054739200)requested by op StatelessRandomUniformV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-07-26 11:39:35.713630: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2022-07-26 11:39:35.713641: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 8, Chunks in use: 8. 2.0KiB allocated for chunks. 2.0KiB in use in bin. 57B client-requested in use in bin.
2022-07-26 11:39:35.713648: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-07-26 11:39:35.713655: I ten

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[1000213825,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

In [ ]:
#Optimizer 
model.compile(optimizer='adam', loss='NLL')

In [ ]:
#Fit
model.fit(features, target, epochs=10, verbose=1)